In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install polars

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import numpy as np
import polars as pl
#import databricks.koalas as ks
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

print('polars version: %s' % pl.__version__)

#print('koalas version: %s' % ks.__version__)

# import dask
# print('dask version: %s' % dask.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 1.4.2
numpy version: 1.21.5
polars version: 0.20.30
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
file_path = "/FileStore/tables/yellow_tripdata_2023_01.parquet"
#pandas_data = pd.read_parquet(file_path, engine="pyarrow")
pandas_data = spark.read.format('parquet').options(header='true').load(file_path).toPandas()

In [0]:
polars_data = pl.from_pandas(pandas_data)
polars_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}
polars_data.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,"""N""",161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.1,1.0,"""N""",43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,"""N""",48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0
1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.9,1.0,"""N""",138,7,1,12.1,7.25,0.5,0.0,0.0,1.0,20.85,0.0,1.25
2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,"""N""",107,79,1,11.4,1.0,0.5,3.28,0.0,1.0,19.68,2.5,0.0


In [0]:
def read_file_parquet():
    return pl.read_parquet("/FileStore/tables/yellow_tripdata_2023_01.parquet")

def count(df):
    return df.height

def count_index_length(df):
    return df.shape[0]

def mean(df):
    return df.select(pl.col("fare_amount").mean()).to_numpy().item()

def standard_deviation(df):
    return df.select(pl.col("fare_amount").std()).to_numpy().item()

def mean_of_sum(df):
    return df.with_columns((pl.col("fare_amount") + pl.col("tip_amount")).alias("sum")).select(pl.col("sum").mean()).to_numpy().item()

def sum_columns(df):
    return df.with_columns((pl.col("fare_amount") + pl.col("tip_amount")).alias("sum"))["sum"]

def mean_of_product(df):
    return df.with_columns((pl.col("fare_amount") * pl.col("tip_amount")).alias("product")).select(pl.col("product").mean()).to_numpy().item()

def product_columns(df):
    return df.with_columns((pl.col("fare_amount") * pl.col("tip_amount")).alias("product"))["product"]

def value_counts(df):
    # return df.select(pl.col("fare_amount")).value_counts()
    return df.groupby("fare_amount").agg(pl.col("fare_amount").count().alias("counts")).sort("counts")

def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df["start_lon"]
    phi_1 = df["start_lat"]
    theta_2 = df["end_lon"]
    phi_2 = df["end_lat"]
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180)**2 +
            np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1 - temp))
    return pl.Series(ret).mean()

def complicated_arithmetic_operation(df):
    theta_1 = df["start_lon"]
    phi_1 = df["start_lat"]
    theta_2 = df["end_lon"]
    phi_2 = df["end_lat"]
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180)**2 +
            np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1 - temp))
    return ret

def groupby_statistics(df):
    result = df.groupby("passenger_count").agg([
        pl.col("fare_amount").mean().alias("fare_amount_mean"),
        pl.col("fare_amount").std().alias("fare_amount_std"),
        pl.col("tip_amount").mean().alias("tip_amount_mean"),
        pl.col("tip_amount").std().alias("tip_amount_std")
    ])
    return result

# other = groupby_statistics(polars_data)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

other = groupby_statistics(polars_data)
# other.columns = ["_".join(col) for col in other.columns]
# other.columns = new_column_names
# other.columns = [e[0]+'_' + e[1] for e in other.columns]

def join_count(df, other):
    # return df.join(other, left_on="index", right_on="index", how="inner").shape[0]
    return df.join(other, on="passenger_count", how="inner").shape[0]

def join_data(df, other):
    # return df.join(other, left_on="index", right_on="index", how="inner")
    return df.join(other, on="passenger_count", how="inner")

<command-1433832980640705>:53: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  result = df.groupby("passenger_count").agg([


In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=polars_data, benchmarks=polars_benchmarks, name='count')
benchmark(count_index_length, df=polars_data, benchmarks=polars_benchmarks, name='count index length')
benchmark(mean, df=polars_data, benchmarks=polars_benchmarks, name='mean')
benchmark(standard_deviation, df=polars_data, benchmarks=polars_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=polars_data, benchmarks=polars_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=polars_data, benchmarks=polars_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=polars_data, benchmarks=polars_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=polars_data, benchmarks=polars_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=polars_data, benchmarks=polars_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=polars_data, benchmarks=polars_benchmarks, name='groupby statistics')
benchmark(join_count, polars_data, benchmarks=polars_benchmarks, name='join count', other=other)
benchmark(join_data, polars_data, benchmarks=polars_benchmarks, name='join', other=other) # cant join

count took: 1.1444091796875e-05 seconds
count index length took: 6.9141387939453125e-06 seconds
mean took: 0.00504302978515625 seconds
standard deviation took: 0.02655029296875 seconds
mean of columns addition took: 0.022109270095825195 seconds
addition of columns took: 0.01756119728088379 seconds
mean of columns multiplication took: 0.02166128158569336 seconds
multiplication of columns took: 0.01741814613342285 seconds
value counts took: 0.08829426765441895 seconds


<command-1433832980640705>:30: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  return df.groupby("fare_amount").agg(pl.col("fare_amount").count().alias("counts")).sort("counts")
<command-1433832980640705>:53: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  result = df.groupby("passenger_count").agg([


groupby statistics took: 0.1468656063079834 seconds
join count took: 1.7630863189697266 seconds
join took: 0.5437631607055664 seconds
Out[5]: 0.5437631607055664

Operations with filtering

In [0]:
expr_filter = (polars_data["tip_amount"] >= 1) & (polars_data["tip_amount"] <= 5)

def filter_data(df):
    return df.filter(expr_filter)
  
polars_filtered = filter_data(polars_data)

In [0]:
benchmark(count, polars_filtered, benchmarks=polars_benchmarks, name='filtered count')
benchmark(count_index_length, polars_filtered, benchmarks=polars_benchmarks, name='filtered count index length')
benchmark(mean, polars_filtered, benchmarks=polars_benchmarks, name='filtered mean')
benchmark(standard_deviation, polars_filtered, benchmarks=polars_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, polars_filtered, benchmarks=polars_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=polars_filtered, benchmarks=polars_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, polars_filtered, benchmarks=polars_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=polars_filtered, benchmarks=polars_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, polars_filtered, benchmarks=polars_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, polars_filtered, benchmarks=polars_benchmarks, name='filtered groupby statistics')

# other = groupby_statistics(polars_filtered)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])
other = groupby_statistics(polars_data)

benchmark(join_count, polars_filtered, benchmarks=polars_benchmarks, name='filtered join count', other=other)
benchmark(join_data, polars_filtered, benchmarks=polars_benchmarks, name='filtered join', other=other)

filtered count took: 1.1920928955078125e-05 seconds
filtered count index length took: 8.821487426757812e-06 seconds
filtered mean took: 0.017596006393432617 seconds
filtered standard deviation took: 0.013469696044921875 seconds
filtered mean of columns addition took: 0.014718055725097656 seconds
filtered addition of columns took: 0.01087045669555664 seconds
filtered mean of columns multiplication took: 0.013991832733154297 seconds
filtered multiplication of columns took: 0.010532617568969727 seconds
filtered value counts took: 0.04606461524963379 seconds


<command-1433832980640705>:30: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  return df.groupby("fare_amount").agg(pl.col("fare_amount").count().alias("counts")).sort("counts")
<command-1433832980640705>:53: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  result = df.groupby("passenger_count").agg([


filtered groupby statistics took: 0.07742929458618164 seconds
filtered join count took: 0.4343430995941162 seconds
filtered join took: 0.37726569175720215 seconds
Out[7]: 0.37726569175720215

In [0]:
polars_res_temp = get_results(polars_benchmarks).set_index('task')
polars_res_temp

,duration
task,
count,0.000011
count index length,0.000007
mean,0.005043
standard deviation,0.026550
mean of columns addition,0.022109
addition of columns,0.017561
mean of columns multiplication,0.021661
multiplication of columns,0.017418
value counts,0.088294
